# Preprocessing & EDA (Tabular + Geospatial)

## Steps
1. Load `train(1).xlsx`, `test2.xlsx`
2. Basic cleaning: parse dates, handle `yr_renovated==0` → NaN, log1p skewed sqft/lot
3. Geo: build GeoDataFrame from lat/long, plot price heatmap
4. Visual samples: show fetched satellite tiles side-by-side with price
5. Correlations: Pearson/Spearman; feature distributions; outlier checks
6. Save cleaned CSV/Parquet for training

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
import sys
sys.path.append('..')
from src.config import cfg

## 1. Load Data

In [ ]:
# Load train and test data
train = pd.read_excel("../data/train(1).xlsx")
test = pd.read_excel("../data/test2.xlsx")

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
train.head()

## 2. Basic Data Info & Statistics

In [ ]:
# Data info
print("=== Train Data Info ===")
train.info()
print("\n=== Missing Values ===")
print(train.isnull().sum())

In [ ]:
# Statistical summary
train.describe()

## 3. Data Cleaning

In [ ]:
# Parse dates and handle yr_renovated
train['date'] = pd.to_datetime(train['date'])

for df in [train, test]:
    # yr_renovated = 0 means never renovated, convert to NaN
    df.loc[df['yr_renovated'] == 0, 'yr_renovated'] = np.nan
    
    # Log transform skewed features
    for col in ['sqft_lot', 'sqft_lot15']:
        df[col + '_log'] = np.log1p(df[col])

print("Data cleaning complete!")
print(f"yr_renovated NaN count: {train['yr_renovated'].isna().sum()}")

## 4. Price Distribution Analysis

In [ ]:
# Price distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original price distribution
axes[0].hist(train['price'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Price')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Price Distribution')

# Log-transformed price
axes[1].hist(np.log1p(train['price']), bins=50, edgecolor='black', alpha=0.7, color='green')
axes[1].set_xlabel('Log(Price)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Log-Transformed Price Distribution')

plt.tight_layout()
plt.show()

## 5. Geospatial Analysis

In [ ]:
# Create GeoDataFrame
gdf = gpd.GeoDataFrame(
    train, 
    geometry=gpd.points_from_xy(train.long, train.lat), 
    crs="EPSG:4326"
)

# Plot price choropleth
fig, ax = plt.subplots(figsize=(10, 8))
gdf.plot(
    column='price', 
    scheme='Quantiles', 
    k=5, 
    legend=True, 
    ax=ax,
    cmap='RdYlGn_r',
    markersize=5,
    alpha=0.6
)
ax.set_title("House Prices by Location (Quantile Classification)")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
plt.show()

## 6. Correlation Analysis

In [ ]:
# Correlation heatmap
numeric_cols = train.select_dtypes(include=[np.number]).columns
corr_matrix = train[numeric_cols].corr()

# Focus on correlations with price
price_corr = corr_matrix['price'].sort_values(ascending=False)
print("Top correlations with price:")
print(price_corr.head(10))

# Heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Feature Correlation Heatmap')
plt.tight_layout()
plt.show()

## 7. Feature Distributions

In [ ]:
# Key feature distributions
key_features = ['sqft_living', 'grade', 'bathrooms', 'bedrooms', 'view', 'condition']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, feat in enumerate(key_features):
    axes[idx].hist(train[feat], bins=30, edgecolor='black', alpha=0.7)
    axes[idx].set_xlabel(feat)
    axes[idx].set_ylabel('Frequency')
    axes[idx].set_title(f'{feat} Distribution')

plt.tight_layout()
plt.show()

## 8. Price vs Key Features

In [ ]:
# Scatter plots: Price vs key features
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

sns.scatterplot(data=train, x='sqft_living', y='price', alpha=0.5, ax=axes[0,0])
axes[0,0].set_title('Price vs Sqft Living')

sns.scatterplot(data=train, x='grade', y='price', alpha=0.5, ax=axes[0,1])
axes[0,1].set_title('Price vs Grade')

sns.boxplot(data=train, x='bedrooms', y='price', ax=axes[1,0])
axes[1,0].set_title('Price by Bedrooms')

sns.boxplot(data=train, x='view', y='price', ax=axes[1,1])
axes[1,1].set_title('Price by View')

plt.tight_layout()
plt.show()

## 9. Save Cleaned Data

In [ ]:
# Save cleaned data
train.to_csv('../data/train_cleaned.csv', index=False)
test.to_csv('../data/test_cleaned.csv', index=False)

print("Cleaned data saved!")
print(f"Train saved to: ../data/train_cleaned.csv")
print(f"Test saved to: ../data/test_cleaned.csv")